<a href="https://colab.research.google.com/github/ArdeshirB/Projects/blob/main/Code%20To%20Be%20Shared%20With%20Anmoy%20-%20Tonnage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import pyodbc
import pandas as pd
from datetime import datetime
import numpy as np
import openpyxl
from openpyxl.styles import PatternFill
from openpyxl.utils import get_column_letter

def highlight_column(worksheet, column, fill):
    for row in range(2, worksheet.max_row + 1):
        cell = worksheet[f'{column}{row}']
        cell.fill = fill

def autosize_columns(worksheet):
    for col in worksheet.columns:
        max_length = 0
        column = col[0].column_letter
        for cell in col:
            cell_length = len(str(cell.value))
            if cell_length > max_length:
                max_length = cell_length
        worksheet.column_dimensions[column].width = max_length + 2


connection_string = """
DSN=DWD;UID=gonzalob;Trusted_Connection=Yes;APP=Microsoft Office;
WSID=BJ0994;DATABASE=DWD_Datamarts;ApplicationIntent=READONLY;
"""
sql_query = """
----- New query introducing a cut-off time defined as parameter in the report
with a as
(Select x.MTCONO, x.MTRIDN, x.MTITNO, x.MTWHLO, x.MTBANO, 
min(case when x.MTTRTM < CONVERT(datetime, '2023-03-27 00:00:00') then convert(datetime,convert(char(10),x.MTTRDT,120))-1 else
convert(datetime,convert(char(10),x.MTTRDT,120)) end) [Calc.Prod.Date],
sum(x.MTTRQT) [MO Received Qty], sum(x.MTCAWE) [MO Received CW]
from DWD2_Staging.dbo.MITTRA x where x.MTWHLO='BJ' and x.MTTTYP=10
group by x.MTCONO, x.MTRIDN, x.MTITNO, x.MTWHLO, x.MTBANO),
b as
(Select x.MTCONO, x.MTITNO, x.MTWHLO, x.MTBANO, sum(x.MTTRQT) [RO'd-in Qty], sum(x.MTCAWE) [RO'd-in CW]
from DWD2_Staging.dbo.MITTRA x where x.MTWHLO='BJ' and x.MTTTYP=40
group by x.MTCONO, x.MTITNO, x.MTWHLO, x.MTBANO),
c as
(Select x.MTCONO, x.MTITNO, x.MTWHLO, x.MTBANO, sum(x.MTTRQT) [RO'd-out Qty], sum(x.MTCAWE) [RO'd-out CW]
from DWD2_Staging.dbo.MITTRA x where x.MTWHLO='BJ' and x.MTTTYP=41
group by x.MTCONO, x.MTITNO, x.MTWHLO, x.MTBANO),
d as
(Select x.MTCONO, x.MTITNO, x.MTWHLO, x.MTBANO, sum(x.MTTRQT) [Inv.Adjusted Qty], sum(x.MTCAWE) [Inv.Adjusted CW]
from DWD2_Staging.dbo.MITTRA x where x.MTWHLO='BJ' and x.MTTTYP=90
group by x.MTCONO, x.MTITNO, x.MTWHLO, x.MTBANO),
e as
(Select x.MTCONO, x.MTITNO, x.MTWHLO, x.MTBANO, sum(x.MTTRQT) [Lot Number Reclass'd Qty], sum(x.MTCAWE) [Lot Number Reclass'd CW]
from DWD2_Staging.dbo.MITTRA x where x.MTWHLO='BJ' and x.MTTTYP=97
group by x.MTCONO, x.MTITNO, x.MTWHLO, x.MTBANO),
f as
(Select x.MTCONO, x.MTITNO, x.MTWHLO, x.MTBANO, sum(x.MTTRQT) [Item Reclass'd Qty], sum(x.MTCAWE) [Item Reclass'd CW]
from DWD2_Staging.dbo.MITTRA x where x.MTWHLO='BJ' and x.MTTTYP=98
group by x.MTCONO, x.MTITNO, x.MTWHLO, x.MTBANO)

Select a.MTRIDN [MO Number], TRIM(a.MTITNO) [Product], TRIM(y.MMITTY) "Item Type", TRIM(y.MMITDS) "Product Name", a.MTBANO "Lot Number",
a.[Calc.Prod.Date], TRIM(y.MMUNMS) "UoM", a.[MO Received Qty], b.[RO'd-in Qty],
c.[RO'd-out Qty], d.[Inv.Adjusted Qty], --e.[Lot Number Reclass'd Qty], f.[Item Reclass'd Qty],
y.MMNEWE "UoM Net Weight", a.[MO Received CW], b.[RO'd-in CW], c.[RO'd-out CW], d.[Inv.Adjusted CW], z.MUCOFA "OM"
from a
inner join DWD2_Staging.dbo.MITMAS y on a.MTCONO=y.MMCONO and a.MTITNO=y.MMITNO
left join DWD_Staging.dbo.MITAUN z on a.MTCONO=z.MUCONO and a.MTITNO=z.MUITNO and z.MUAUTP=1 and z.MUALUN='PCK'
left join b on a.MTCONO=b.MTCONO and a.MTWHLO=b.MTWHLO and a.MTITNO=b.MTITNO and a.MTBANO=b.MTBANO
left join c on a.MTCONO=c.MTCONO and a.MTWHLO=c.MTWHLO and a.MTITNO=c.MTITNO and a.MTBANO=c.MTBANO
left join d on a.MTCONO=d.MTCONO and a.MTWHLO=d.MTWHLO and a.MTITNO=d.MTITNO and a.MTBANO=d.MTBANO
--left join e on a.MTCONO=e.MTCONO and a.MTWHLO=e.MTWHLO and a.MTITNO=e.MTITNO and a.MTBANO=e.MTBANO
--left join f on a.MTCONO=f.MTCONO and a.MTWHLO=f.MTWHLO and a.MTITNO=f.MTITNO and a.MTBANO=f.MTBANO
where y.MMITTY in ('Z30','Z31','Z40','Z41','Z60')
order by a.[Calc.Prod.Date], a.MTRIDN, a.MTBANO
"""

# Replace the placeholder with the actual cut-off time value
cut_off_time = datetime.now().strftime("'%Y-%m-%d %H:%M:%S'")
sql_query = sql_query.replace('?', cut_off_time)



# Connect to the database and run the query
connection = pyodbc.connect(connection_string)
df = pd.read_sql_query(sql_query, connection)
connection.close()


# Replace NaN values with 0
df.fillna(0, inplace=True)

# Add the 'Estimated Produced Qty' column
df['Estimated Produced Qty'] = df['MO Received Qty'] + df['RO\'d-in Qty'] + df['RO\'d-out Qty'] + df['Inv.Adjusted Qty']

# Add the 'Estimated Produced KG' column
df['Estimated Produced KG'] = np.where(df['Item Type'] == 'Z41', df['MO Received CW'] + df['RO\'d-in CW'] + df['RO\'d-out CW'] + df['Inv.Adjusted CW'], df['Estimated Produced Qty'] * df['UoM Net Weight'])

# Add the 'Estimated Produced PCK' column
df['Estimated Produced PCK'] = df['Estimated Produced Qty'] * df['OM']

# Convert the 'Calc.Prod.Date' column to datetime
df['Calc.Prod.Date'] = pd.to_datetime(df['Calc.Prod.Date'])
df = df[df['Calc.Prod.Date'] > '2023-01-01']

FG_query = pd.read_excel('O:\Ardy\Tonnage\Relations\PLU-VS Relation FG.xlsx')
FG_query['Product'] = FG_query['Product'].astype(str)
FG_merged_df = df.merge(FG_query[['Product', 'Value Stream']], on='Product', how='left')


# Save the result to an Excel file
FG_merged_df.to_excel('FG.xlsx', index=False)

FG_merged_df['Date'] = FG_merged_df['Calc.Prod.Date'].dt.date
FG_grouped_df = FG_merged_df.groupby(['Value Stream', 'Date']).agg({'Estimated Produced KG': 'sum'})
FG_pivoted_df = FG_grouped_df.pivot_table(index='Value Stream', columns='Date', values=['Estimated Produced KG'])
FG_pivoted_df.columns = [f'{col[1].strftime("%d/%m/%Y")}' for col in FG_pivoted_df.columns]

filename_pivot = 'FG_Pivot.xlsx'
FG_pivoted_df.to_excel(filename_pivot)

# Open the Excel file with openpyxl
workbook_pivot = openpyxl.load_workbook(filename_pivot)
worksheet_pivot = workbook_pivot.active
light_blue_fill = PatternFill(start_color="ADD8E6", end_color="ADD8E6", fill_type="solid")
yellow_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
# Apply the light blue background color to the first row
for cell in worksheet_pivot["1:1"]:
    cell.fill = light_blue_fill

# Save the changes to the Excel file
workbook_pivot.save(filename_pivot)
highlight_column(worksheet_pivot, "A", yellow_fill)
autosize_columns(worksheet_pivot)

# Save the changes to the Excel file
workbook_pivot.save(filename_pivot)
workbook_pivot.close()


connection_string = """
DSN=DWD;UID=gonzalob;Trusted_Connection=Yes;APP=Microsoft Office;WSID=BJ0994;DATABASE=DWD_Datamarts;ApplicationIntent=READONLY;
"""
sql_query = """
----- New query introducing a cut-off time defined as parameter in the report
with a as
(Select x.MTCONO, x.MTRIDN, x.MTITNO, x.MTWHLO, x.MTBANO, 
min(case when x.MTTRTM < CAST('2023-03-27 13:08:53' as datetime) then convert(datetime,convert(char(10),x.MTTRDT,120))-1 else
convert(datetime,convert(char(10),x.MTTRDT,120)) end) [Calc.Prod.Date],
sum(x.MTTRQT) [MO Received Qty]
from DWD2_Staging.dbo.MITTRA x where x.MTWHLO='BJ' and x.MTTTYP=10
group by x.MTCONO, x.MTRIDN, x.MTITNO, x.MTWHLO, x.MTBANO),
b as
(Select x.MTCONO, x.MTITNO, x.MTWHLO, x.MTBANO, sum(x.MTTRQT) [RO'd-in Qty]
from DWD2_Staging.dbo.MITTRA x where x.MTWHLO='BJ' and x.MTTTYP=40
group by x.MTCONO, x.MTITNO, x.MTWHLO, x.MTBANO),
c as
(Select x.MTCONO, x.MTITNO, x.MTWHLO, x.MTBANO, sum(x.MTTRQT) [RO'd-out Qty]
from DWD2_Staging.dbo.MITTRA x where x.MTWHLO='BJ' and x.MTTTYP=41
group by x.MTCONO, x.MTITNO, x.MTWHLO, x.MTBANO),
d as
(Select x.MTCONO, x.MTITNO, x.MTWHLO, x.MTBANO, sum(x.MTTRQT) [Inv.Adjusted Qty]
from DWD2_Staging.dbo.MITTRA x where x.MTWHLO='BJ' and x.MTTTYP=90
group by x.MTCONO, x.MTITNO, x.MTWHLO, x.MTBANO),
e as
(Select x.MTCONO, x.MTITNO, x.MTWHLO, x.MTBANO, sum(x.MTTRQT) [Lot Number Reclass'd Qty]
from DWD2_Staging.dbo.MITTRA x where x.MTWHLO='BJ' and x.MTTTYP=97
group by x.MTCONO, x.MTITNO, x.MTWHLO, x.MTBANO),
f as
(Select x.MTCONO, x.MTITNO, x.MTWHLO, x.MTBANO, sum(x.MTTRQT) [Item Reclass'd Qty]
from DWD2_Staging.dbo.MITTRA x where x.MTWHLO='BJ' and x.MTTTYP=98
group by x.MTCONO, x.MTITNO, x.MTWHLO, x.MTBANO)

Select a.MTRIDN [MO Number], TRIM(a.MTITNO) [Product], TRIM(y.MMITTY) "Item Type", TRIM(y.MMITDS) "Product Name", a.MTBANO "Lot Number",
[Calc.Prod.Date], TRIM(y.MMUNMS) "UoM",
case when y.MMUNMS in ('KG') then a.[MO Received Qty] else 0 end "MO Received KGs",
case when y.MMUNMS in ('KG') then b.[RO'd-in Qty] else 0 end "RO'd-in KGs",
case when y.MMUNMS in ('KG') then c.[RO'd-out Qty] else 0 end "RO'd-out KGs",
case when y.MMUNMS in ('KG') then d.[Inv.Adjusted Qty] else 0 end "Inv.Adjusted KGs",
y.MMNEWE "UoM Net Weight",
case when y.MMUNMS in ('CTN','CRA') then a.[MO Received Qty] else 0 end "MO Received CTNs",
case when y.MMUNMS in ('CTN','CRA') then b.[RO'd-in Qty] else 0 end "RO'd-in CTNs",
case when y.MMUNMS in ('CTN','CRA') then c.[RO'd-out Qty] else 0 end "RO'd-out CTNs",
case when y.MMUNMS in ('CTN','CRA') then d.[Inv.Adjusted Qty] else 0 end "Inv.Adjusted CTNs"
from a
inner join DWD2_Staging.dbo.MITMAS y on a.MTCONO=y.MMCONO and a.MTITNO=y.MMITNO
left join b on a.MTCONO=b.MTCONO and a.MTWHLO=b.MTWHLO and a.MTITNO=b.MTITNO and a.MTBANO=b.MTBANO
left join c on a.MTCONO=c.MTCONO and a.MTWHLO=c.MTWHLO and a.MTITNO=c.MTITNO and a.MTBANO=c.MTBANO
left join d on a.MTCONO=d.MTCONO and a.MTWHLO=d.MTWHLO and a.MTITNO=d.MTITNO and a.MTBANO=d.MTBANO
--left join e on a.MTCONO=e.MTCONO and a.MTWHLO=e.MTWHLO and a.MTITNO=e.MTITNO and a.MTBANO=e.MTBANO
--left join f on a.MTCONO=f.MTCONO and a.MTWHLO=f.MTWHLO and a.MTITNO=f.MTITNO and a.MTBANO=f.MTBANO
where y.MMITTY in ('Z32','Z42','Z38')
order by a.[Calc.Prod.Date], a.MTRIDN, a.MTBANO
"""

# Replace the placeholder with the actual cut-off time value
cut_off_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
sql_query = sql_query.replace('?', cut_off_time)


# Connect to the database and run the query
connection = pyodbc.connect(connection_string)
df = pd.read_sql_query(sql_query, connection)
connection.close()

# Replace NaN values with 0
df.fillna(0, inplace=True)

# Add the 'Estimated Produced CTNs' column
df['Estimated Produced CTNs'] = df['MO Received CTNs'] 

# Add the 'Estimated Produced KG' column
df['Estimated Produced KG'] = np.where(df['UoM'] == 'KG', df['MO Received KGs'] , df['Estimated Produced CTNs'] * df['UoM Net Weight'])

# Convert the 'Calc.Prod.Date' column to datetime
df['Calc.Prod.Date'] = pd.to_datetime(df['Calc.Prod.Date'])

# Filter rows based on the date condition
df = df[df['Calc.Prod.Date'] > '2023-01-01']


WIP_query = pd.read_excel('O:\Ardy\Tonnage\Relations\PLU-VS Relation WIP.xlsx')
WIP_query['Product'] = WIP_query['Product'].astype(str)
WIP_merged_df = df.merge(WIP_query[['Product', 'Value Stream']], on='Product', how='left')

# Save the result to an Excel file
WIP_merged_df.to_excel('WIP_SFG.xlsx', index=False)

WIP_merged_df['Date'] = WIP_merged_df['Calc.Prod.Date'].dt.date
grouped_df = WIP_merged_df.groupby(['Value Stream', 'Date']).agg({'Estimated Produced KG': 'sum'})
WIP_pivoted_df = grouped_df.pivot_table(index='Value Stream', columns='Date', values=['Estimated Produced KG'])
WIP_pivoted_df.columns = [f'{col[1].strftime("%d/%m/%Y")}' for col in WIP_pivoted_df.columns]


filename_pivot = 'WIP_SFG_Pivot.xlsx'
WIP_pivoted_df.to_excel(filename_pivot)

# Open the Excel file with openpyxl
workbook_pivot = openpyxl.load_workbook(filename_pivot)
worksheet_pivot = workbook_pivot.active
light_blue_fill = PatternFill(start_color="ADD8E6", end_color="ADD8E6", fill_type="solid")
yellow_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
# Apply the light blue background color to the first row
for cell in worksheet_pivot["1:1"]:
    cell.fill = light_blue_fill

# Save the changes to the Excel file
workbook_pivot.save(filename_pivot)
highlight_column(worksheet_pivot, "A", yellow_fill)
autosize_columns(worksheet_pivot)

# Save the changes to the Excel file
workbook_pivot.save(filename_pivot)
workbook_pivot.close()


merged_df = pd.concat([FG_pivoted_df, WIP_pivoted_df])
merged_df_summed = merged_df.groupby(merged_df.index).sum()
merged_df_summed.to_excel('Merged_Pivot.xlsx')
filename_merged = 'Merged_Pivot.xlsx'
workbook_merged = openpyxl.load_workbook(filename_merged)
worksheet_merged = workbook_merged.active

for cell in worksheet_merged["1:1"]:
    cell.fill = light_blue_fill

highlight_column(worksheet_merged, "A", yellow_fill)
autosize_columns(worksheet_merged)

workbook_merged.save(filename_merged)
workbook_merged.close()


# Read the Excel files
df_fg = pd.read_excel("FG_Pivot.xlsx", engine='openpyxl')
df_wip = pd.read_excel("WIP_SFG_Pivot.xlsx", engine='openpyxl')
df_merged = pd.read_excel("Merged_Pivot.xlsx", engine='openpyxl')


import numpy as np

def cell_style(value):
    if pd.isna(value):
        return 'display: none'
    return ''

def date_highlight(s):
    is_date = str(s.name).lower() == 'date'
    return ['background-color: lightblue' if is_date else '' for _ in s]

def value_stream_highlight(s):
    is_value_stream = str(s.name).lower() == 'value stream'
    return ['background-color: yellow' if is_value_stream else '' for _ in s]

def highlight_dates_and_value_streams(df):
    df.fillna('', inplace=True)
    styled_df = (
        df.style
        .apply(date_highlight, axis=1)
        .apply(value_stream_highlight, axis=1)
        .format(lambda x: '' if x == '' else "{:,.0f}".format(x) if isinstance(x, (int, float)) else x, na_rep="")

    )
    return styled_df.applymap(cell_style).render()


fg_html_table = highlight_dates_and_value_streams(df_fg)
wip_html_table = highlight_dates_and_value_streams(df_wip)
merged_html_table = highlight_dates_and_value_streams(df_merged)




html_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Data Tables</title>
    <style>
        .tab button {{
            display: inline-block;
            background-color: inherit;
            border: none;
            outline: none;
            cursor: pointer;
            padding: 14px 16px;
            transition: 0.3s;
            font-size: 17px;
        }}
        .tab button:hover {{
            background-color: #ddd;
        }}
        .tab button.active {{
            background-color: #ccc;
        }}
        .tabcontent {{
            display: none;
            padding: 6px 12px;
            border: 1px solid #ccc;
            border-top: none;
        }}
    </style>
</head>
<body>
    <div class="tab">
        <button class="tablinks" onclick="openTab(event, 'FG')">FG</button>
        <button class="tablinks" onclick="openTab(event, 'WIP')">WIP</button>
        <button class="tablinks" onclick="openTab(event, 'Merged')">Merged</button>
    </div>

    <div id="FG" class="tabcontent">
        {fg_html_table}
    </div>
    <div id="WIP" class="tabcontent">
        {wip_html_table}
    </div>
    <div id="Merged" class="tabcontent">
        {merged_html_table}
    </div>

    <script>
        function openTab(evt, tabName) {{
            var i, tabcontent, tablinks;
            tabcontent = document.getElementsByClassName("tabcontent");
            for (i = 0; i < tabcontent.length; i++) {{
                tabcontent[i].style.display = "none";
            }}
            tablinks = document.getElementsByClassName("tablinks");
            for (i = 0; i < tablinks.length; i++) {{
                tablinks[i].className = tablinks[i].className.replace(" active", "");
            }}
            document.getElementById(tabName).style.display = "block";
            evt.currentTarget.className += " active";
        }}
        document.getElementsByClassName("tablinks")[0].click();
    </script>
</body>
</html>
'''


with open("output.html", "w") as f:
    f.write(html_template.format(fg_html_table=fg_html_table, wip_html_table=wip_html_table, merged_html_table=merged_html_table))



